### Notebook responsável por executar os diversos notebooks

In [1]:
%run ./database.ipynb
%run ./clubes.ipynb
%run ./partidas.ipynb
%run ./jogadores.ipynb
%run ./transformacoes.ipynb

In [2]:
def __busca_datasets_banco_dados__():
    engine = prepare_database()
    
    rows = collect_clubs_and_save_to_database(engine)
    print("Quantidade de linhas para clubes: {}".format(rows))
    
    rows = collect_matches_and_save_to_database(engine)
    print("Quantidade de linhas para partidas: {}".format(rows))
    
    rows = collect_scouts_and_save_to_database(engine)
    print("Quantidade de linhas para scouts: {}".format(rows))
    
    pre_df_matches = get_matches_dataframe(engine)
    pre_df_scouts = get_scouts_dataframe(engine)
    df_clubs = get_clubs_dataframe(engine)
    
    return pre_df_matches, pre_df_scouts, df_clubs

In [1]:
def __transforma_dataframes__(pre_df_matches, pre_df_scouts, df_clubs):
    df_matches = aplica_transformacoes_dataframe_partidas(pre_df_matches)
    print(medida_casa_empate_fora(df_matches))
    
    df_scouts = realiza_merge_entre_partidas_scouts_clube(df_matches, pre_df_scouts, df_clubs)
    df_pontuacao = cria_dataframe_pontuacao(df_scouts)
    
    df_mandante_x_visitante = cria_dataframe_mandante_x_visitante(df_scouts, df_clubs)
    df_mandante_x_visitante_cedidos_mais_conquistados = cria_dataframe_cedidos_mais_conquistados(df_mandante_x_visitante)
    
    df_visitante_x_mandante = cria_dataframe_visitante_x_mandante(df_scouts, df_clubs)
    df_visitante_x_mandante_cedidos_mais_conquistados = cria_dataframe_cedidos_mais_conquistados(df_visitante_x_mandante)
    
    df_conquistados = cria_dataframe_pontuacao_conquistada(df_scouts, df_clubs)
    df_cedidos = cria_dataframe_pontuacao_cedida(df_scouts, df_clubs)
    
    df_matches.columns = ['match_id','round','home_id','visitor_id','date','timestamp','local','valid','home_goal','visitor_goal','year','diff_goals','vencedor','datetime']

    df_club_matches, df_club_matches_as_mandante, df_club_matches_as_visitor = cria_dataframe_estatisticas_gols_acumulados_por_time(df_matches, df_clubs)

    df_club_matches_as_visitor.drop(['ref_id', 'ref_goals_scored_as_visitor', 'ref_goals_against_as_visitor', 'ref_as_visitor_result'], axis=1, inplace=True)
    df_club_matches_as_visitor.columns=['match_id', 'round', 'home_id', 'visitor_id', 'local', 'valid',\
           'home_goal', 'visitor_goal', 'year', 'diff_goals', 'vencedor',\
           'datetime', 'goals_scored_as_visitor_sum',\
           'goals_against_as_visitor_sum', 'goals_as_visitor_diff']
    df_club_matches_as_mandante.drop(['ref_id', 'ref_goals_scored_as_mandante', 'ref_goals_against_as_mandante', 'ref_as_mandante_result'], axis=1, inplace=True)
    df_club_matches_as_mandante.columns=['match_id', 'round', 'home_id', 'visitor_id', 'local', 'valid',\
           'home_goal', 'visitor_goal', 'year', 'diff_goals', 'vencedor',\
           'datetime', 'goals_scored_as_mandante_sum',\
           'goals_against_as_mandante_sum', 'goals_as_mandante_diff']
    df_club_matches_as_mandante_visitor = pd.merge(df_club_matches_as_mandante, df_club_matches_as_visitor, on=['match_id', 'round', 'home_id', 'visitor_id', 'local', 'valid',\
           'home_goal', 'visitor_goal', 'year', 'diff_goals', 'vencedor',\
           'datetime'], how='inner')    
    
    df_club_matches_home_visitor = cria_dataframe_estatisticas_gols_acumulados_mandante_visitante(df_club_matches)

    ##TODO retomar daqui
    return df_matches, df_scouts, df_pontuacao, df_mandante_x_visitante, df_mandante_x_visitante_cedidos_mais_conquistados, df_visitante_x_mandante, df_visitante_x_mandante_cedidos_mais_conquistados, df_conquistados, df_cedidos, df_club_matches, df_club_matches_home_visitor, df_club_matches_as_mandante_visitor

In [2]:
def prepara_dataframes():
    pre_df_matches, pre_df_scouts, df_clubs = __busca_datasets_banco_dados__()
    df_matches, df_scouts, df_pontuacao, df_mandante_x_visitante, df_mandante_x_visitante_cedidos_mais_conquistados, df_visitante_x_mandante, df_visitante_x_mandante_cedidos_mais_conquistados, df_conquistados, df_cedidos, df_club_matches, df_club_matches_home_visitor, df_club_matches_as_mandante_visitor = __transforma_dataframes__(pre_df_matches, pre_df_scouts, df_clubs)
    return df_matches, df_scouts, df_pontuacao, df_clubs, df_mandante_x_visitante, df_mandante_x_visitante_cedidos_mais_conquistados, df_visitante_x_mandante, df_visitante_x_mandante_cedidos_mais_conquistados, df_conquistados, df_cedidos, df_club_matches, df_club_matches_home_visitor, df_club_matches_as_mandante_visitor